# Optimising functions in F# #

This is an appendix to the main F# notebook, concerning writing an optimised [2-sample KS test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test). It is complete, but the aim here is to describe what was done, what worked (and what did not), and to highlight some features of the language that can be used to improve performance.

The fundamental loop here was

1. Benchmark the function
2. Benchmark lines in the function to see where most of the work is done
3. Replace lines and goto 1.

There are not many references for optimising F# code, particularly for programmers who do not have a deep understanding of underpinning features of the language/compiler. However, I strongly recommend [F# for scientists](https://dl.acm.org/citation.cfm?id=1481410) as its description on the process of considering different aspects of optimising scientfic code. Additionally, the [F# journal](http://www.ffconsultancy.com/products/fsharp_journal/), has a number of articles that touch on optimisation and specifically the [Quicksort](http://www.ffconsultancy.com/products/fsharp_journal/subscribers/quicksort.html) article gives an excellent description of language features.

In [3]:
//This is where your python 3.5 install is.
//Macports on a mac may install it at /opt/local/Library/Frameworks/Python.framework/Versions/3.5/
let pythonPath = @"C:\Users\bh418\AppData\Local\Programs\Python\Python35\"
//This is where your notebook is. This cannot be found automatically and is needed to find the supplied experimental data
let dataLocation= sprintf "%s/" (System.IO.Directory.GetCurrentDirectory ()) 

In [4]:
open System
#load "Paket.fsx"
Paket.Package
  [ "MathNet.Numerics"
    "MathNet.Numerics.FSharp"
    "FSharp.Data"
    "Accord"
    "Accord.Statistics"
    "Accord.Math"
    "pythonnet"
    "FSharp.Interop.Dynamic"
  ]
#load "Paket.Generated.Refs.fsx"
open Python.Runtime
open FSharp.Interop.Dynamic
open System.Collections.Generic

//Set a few additional variables that depend on both libraries being loaded and machine specific variables
let pythonLib = pythonPath + @"\lib"
let path' = pythonPath + ";" + Environment.GetEnvironmentVariable("PATH")
Environment.SetEnvironmentVariable("PATH", path')
Environment.SetEnvironmentVariable("PYTHONHOME", pythonPath)
Environment.SetEnvironmentVariable("PYTHONPATH ", pythonLib)

In [5]:
let timeIt f =
   let proc = System.Diagnostics.Process.GetCurrentProcess()
   let stopWatch = System.Diagnostics.Stopwatch.StartNew()
   let t = proc.TotalProcessorTime
   let result = f ()
   let dt = proc.TotalProcessorTime-t
   stopWatch.Stop()
   printfn "Wall: %fms CPU: %fms" stopWatch.Elapsed.TotalMilliseconds dt.TotalMilliseconds
   result

let timeStore r f =
  let proc = System.Diagnostics.Process.GetCurrentProcess()
  let stopWatch = System.Diagnostics.Stopwatch.StartNew()
  let result = f ()
  stopWatch.Stop()
  r := !r + stopWatch.Elapsed.TotalMilliseconds
  result

## Data for benchmarking

As in the previous notebook we read in the experimental data. Alongside this we also generate some simulated datasets for testing, using a normally distributed random numbers around different means (*fakeData1* and *fakeData2*).

In [6]:
let getVal s = if s = "" then nan else float s

let readData (filename: string) count = 
    let raw = FSharp.Data.CsvFile.Load(filename)
    let a = raw.Rows |> Array.ofSeq 
    Array.init (count*(Array.length a)) (fun i -> getVal a.[i/count].[i%count] ) |> Array.filter (fun a ->  not (Double.IsNaN(a)) )

let i0File = dataLocation + "eso_0day.txt"

let i0 = readData i0File 3

let measurements = Array.map (fun (f,c) -> readData f c) [|
                                                            (dataLocation + "eso_7day.txt",3);
                                                            (dataLocation + "eso_12day.txt",3);
                                                            (dataLocation + "eso_18day.txt",2);
                                                            |]
                 |> Array.map (fun m -> Array.map (fun i -> Math.Log(i,2.) ) m)

In [7]:
let fakeDataGenerate rng mean sd = 
    Array.init 1000 (fun _ -> MathNet.Numerics.Distributions.Normal.Sample(rng,mean,sd) )

let rng = System.Random(2020)
let fakeData1 = Array.init 100 (fun _ -> Array.init 3 (fun _ -> fakeDataGenerate rng 0. 1.) )
let fakeData2 = Array.init 100 (fun _ -> Array.init 3 (fun _ -> fakeDataGenerate rng 0. 2.) )

# Original functions

In [8]:
let distance (summStat: 'a [] -> 'a [] -> float) sim measurements = 
    Array.map2 (fun s o -> summStat s o )  sim measurements
    |> Array.sum
    
let ksSummStat s o =
    Accord.Statistics.Testing.TwoSampleKolmogorovSmirnovTest(s,o).Statistic
    
let (scipyKS: float [] -> float [] -> float)  =
    let SeqToPyFloat ( aSeq : float seq ) =
            let list = new Python.Runtime.PyList()
            aSeq |> Seq.iter( fun x -> list.Append( new PyFloat(x)))
            list
    PythonEngine.Initialize()
    ignore <| PythonEngine.BeginAllowThreads()
    fun a b ->
        //use gil = Py.GIL()
        let token = PythonEngine.AcquireLock ()
        let spstats = Py.Import("scipy.stats")
        let np = Py.Import("numpy")
        let npa = np?array( a  |> SeqToPyFloat )
        let npb = np?array( b  |> SeqToPyFloat )
        let r = spstats?ks_2samp(npa,npb)
        let result = r?statistic?item() : float
        PythonEngine.ReleaseLock token
        result

In [9]:
let inline findPositionTS cmp (someArray: _ []) value =
   let rec find lower upper  =
       if lower<upper then
           let div = lower + ((upper-lower) >>> 1)
           let found = someArray.[div]
           if cmp found value > 0 then
               find lower div
           else
               find (div+1) upper
       else
           lower
   find 0 (Array.length someArray)

let inline kscdfTS' cmp (small1: _ []) (small2: _ []) (big: _ []) = 
    let mutable max =  0.
    Array.iter (fun e -> 
        let temp = abs((findPositionTS cmp small1 e |> fun s -> float(s)/float((Array.length small1))) - (findPositionTS cmp small2 e |> fun s -> float(s)/float((Array.length small2))))
        if temp > max then max <- temp
        )  big 
    max

let inline simplifiedKS cmp s o =
    let data1 = Array.sort s //1%
    let data2 = Array.sort o
    let dataAll = Array.append data1 data2 //0.1%
    kscdfTS' cmp data1 data2 dataAll

## Initial benchmarks

These benchmarks demonstrate the issue in our  to 50 times as long

In [10]:
printf "Accord KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance ksSummStat measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance ksSummStat fakeData1.[i] fakeData2.[i]

printf "Scipy KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance scipyKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance scipyKS fakeData1.[i] fakeData2.[i]


Accord KS calculation
Wall: 16625.516300ms CPU: 16453.125000ms
Wall: 2893.382100ms CPU: 2875.000000ms
Scipy KS calculation
Wall: 4284.953400ms CPU: 1687.500000ms
Wall: 581.243800ms CPU: 609.375000ms


# Writing a KS test

Given the performance difference, one obvious solution might be to just use scipy through F#. However, this was suboptimal as running the scipy ks test safely in a multithreaded way is tricky. 

The [algorithm](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test#Two-sample_Kolmogorov%E2%80%93Smirnov_test) we looked to implement takes two unsorted arrays as input

1. Sort the arrays
2. Calculate the the empirical distribution functions of each array
3. Calculate the differences between the empirical distribution functions
4. Find the maximum absolute difference between the two EDFs, and return as the KS statistic

To calculate the empirical distribution function of one sorted array of data relative to another

1. For each element of one array, find the index of the second array where the element could be inserted without breaking the sort
2. Divide the value by the length of the second array

Our first version uses a naive implementation of the process of finding the insertion position

In [11]:
let findPosition someArray value =
    //Find the location to insert the value without breaking the sort
    try 
        Array.findIndex (fun e -> e>value) someArray
    with 
        _ -> (Array.length someArray) 

let cdfDifference a b = 
    let aLen = Array.length a |> float
    let bLen = Array.length b |> float
    Array.mapi (fun i e ->  findPosition a e 
                            |> fun s -> abs ( float(s)/aLen - float(i+1)/bLen) 
                            )  b

let simplifiedKS s o =
    let data1 = Array.sort s
    let data2 = Array.sort o
    let max1 = cdfDifference data1 data2 |> Array.max
    let max2 = cdfDifference data2 data1 |> Array.max
    if max1 > max2 then max1 else max2

In [12]:
printf "Accord: %f\n" <| ksSummStat measurements.[0] measurements.[1]
printf "scipy:  %f\n" <| scipyKS measurements.[0] measurements.[1]
printf "Local:  %f\n" <| simplifiedKS measurements.[0] measurements.[1]

Accord: 0.960469
scipy:  0.960469
Local:  0.960469


In [13]:
printf "Accord KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance ksSummStat measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance ksSummStat fakeData1.[i] fakeData2.[i]

printf "Scipy KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance scipyKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance scipyKS fakeData1.[i] fakeData2.[i]

printf "Local KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance simplifiedKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance simplifiedKS fakeData1.[i] fakeData2.[i]


Accord KS calculation
Wall: 16018.786400ms CPU: 16250.000000ms
Wall: 2780.799500ms CPU: 2781.250000ms
Scipy KS calculation
Wall: 748.678100ms CPU: 843.750000ms
Wall: 484.341300ms CPU: 531.250000ms
Local KS calculation
Wall: 67644.073000ms CPU: 68062.500000ms
Wall: 11009.406300ms CPU: 10812.500000ms


# The new function is 10x slower!

Not a great start! This is however perhaps not surprising as we have simply picked the easily available functions that work. To find out where the function is spending its time we can use the timeStore function defined above with a modified version of the function. We store the time taken taken to sort the arrays and time taken to calculate the cdf to find where the bottleneck is.

In [14]:
let sortTime = ref 0.
let cdfTime = ref 0.

let simplifiedKSTimed s o =
    let data1 = timeStore sortTime (fun _ -> Array.sort s)
    let data2 = timeStore sortTime (fun _ -> Array.sort o)
    let max1 = timeStore cdfTime (fun _ -> cdfDifference data1 data2 |> Array.max)
    let max2 = timeStore cdfTime (fun _ -> cdfDifference data2 data1 |> Array.max)
    if max1 > max2 then max1 else max2
    
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance simplifiedKSTimed measurements measurements)
printf "sort: %f\t cdf: %f\n" !sortTime !cdfTime

Wall: 62122.468900ms CPU: 61453.125000ms
sort: 67.309800	 cdf: 62050.148300


# Bottleneck: calculating the empirical CDF

Almost the entire time is spent calculating CDFs, so replacing the findPosition function is the first feature to optimise. A binary search would be faster so an initial replacement *findPosition* is written based on that.

Our first implementation uses an internal recursive loop to search through the list by iteratively selecting smaller bounds until either we are examining two adjacent integers, or the value midway between the bounds is equal to the target. If these conditions are made an index is returned. Alternatively, depending on whether the midway value is higher or lower than the input value we replace either the lower or upper index with the midway index and restart the loop.

In [15]:
let findPosition (someArray: 'a []) value =
    let rec find lower upper  =
        let dt = (upper-lower)/2
        let div = dt + lower
        if dt = 0 then 
            if someArray.[div]<value then div+1 else div
        elif someArray.[div] = value then div + 1
        elif someArray.[div] > value then find lower div 
        else find div upper 
    find 0 (Array.length someArray) 
    
let cdfDifference a b = 
    let aLen = Array.length a |> float
    let bLen = Array.length b |> float
    Array.mapi (fun i e ->  findPosition a e 
                            |> fun s -> abs ( float(s)/aLen - float(i+1)/bLen) 
                            )  b

let simplifiedKS s o =
    let data1 = Array.sort s
    let data2 = Array.sort o
    let max1 = cdfDifference data1 data2 |> Array.max
    let max2 = cdfDifference data2 data1 |> Array.max
    if max1 > max2 then max1 else max2

In [16]:
printf "Accord KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance ksSummStat measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance ksSummStat fakeData1.[i] fakeData2.[i]

printf "Scipy KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance scipyKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance scipyKS fakeData1.[i] fakeData2.[i]

printf "Local KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance simplifiedKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance simplifiedKS fakeData1.[i] fakeData2.[i]

Accord KS calculation
Wall: 16524.745000ms CPU: 16437.500000ms
Wall: 2978.598900ms CPU: 2968.750000ms
Scipy KS calculation
Wall: 754.365900ms CPU: 828.125000ms
Wall: 549.742300ms CPU: 703.125000ms
Local KS calculation
Wall: 6453.394800ms CPU: 7062.500000ms
Wall: 440.195600ms CPU: 453.125000ms


In [17]:
printf "Accord: %f\n" <| ksSummStat measurements.[0] measurements.[1]
printf "scipy:  %f\n" <| scipyKS measurements.[0] measurements.[1]
printf "Local:  %f\n" <| simplifiedKS measurements.[0] measurements.[1]

Accord: 0.960469
scipy:  0.960469
Local:  0.960469


# Writing a more compact *findPosition*

The new findPosition *is* correct for our simple test, and now faster than the Accord function, but still substantially slower than scipy. Can we improve on that by simplifying the code? A short rewrite can reduce the number of variables created and possibly offer some additional speed.

Now we test if upper and lower are equal once per loop- if they are equal we return their value. In the rest of the loop we find the midway index, and update the bounds as above based on a comparison with the input value.

In [18]:
let findPosition (someArray: 'a []) value =
    let rec find lower upper  =
        if lower=upper then upper else
            let div = (upper-lower)/2 + lower    
            if someArray.[div] > value then find lower div 
            else find (div+1) upper 
    find 0 (Array.length someArray) 
    
let cdfDifference a b = 
    let aLen = Array.length a |> float
    let bLen = Array.length b |> float
    Array.mapi (fun i e ->  findPosition a e 
                            |> fun s -> abs ( float(s)/aLen - float(i+1)/bLen) 
                            )  b

let simplifiedKS s o =
    let data1 = Array.sort s
    let data2 = Array.sort o
    let max1 = cdfDifference data1 data2 |> Array.max
    let max2 = cdfDifference data2 data1 |> Array.max
    if max1 > max2 then max1 else max2

In [19]:
printf "Accord KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance ksSummStat measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance ksSummStat fakeData1.[i] fakeData2.[i]

printf "Scipy KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance scipyKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance scipyKS fakeData1.[i] fakeData2.[i]

printf "Local KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance simplifiedKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance simplifiedKS fakeData1.[i] fakeData2.[i]

Accord KS calculation
Wall: 15677.636900ms CPU: 15671.875000ms
Wall: 2829.169600ms CPU: 2828.125000ms
Scipy KS calculation
Wall: 727.505200ms CPU: 828.125000ms
Wall: 490.260500ms CPU: 546.875000ms
Local KS calculation
Wall: 5893.325100ms CPU: 6687.500000ms
Wall: 327.718500ms CPU: 406.250000ms


In [20]:
printf "Accord: %f\n" <| ksSummStat measurements.[0] measurements.[1]
printf "scipy:  %f\n" <| scipyKS measurements.[0] measurements.[1]
printf "Local:  %f\n" <| simplifiedKS measurements.[0] measurements.[1]

Accord: 0.960469
scipy:  0.960469
Local:  0.960469


## Tree based approach

One alternative approach I explored was to convert the array to a tree structure and search this. The performance (see below) was not good, and I abandoned it ultimately. However, it did raise an interesting performance trade-off. Constructing the tree was too slow for this to be useful, though searching was fast. In our situation however, we reuse one set of arrays repeatedly- the experimental observations. In principle, it may be possible to get a further improvement in speed based on calculating the tree once for the experimental observations and repeatedly using this search approach, whilst using a traditional approach for simulated data. Here we have included the tree based code for general interest.

In [21]:
type locationLookup<'a> = Leaf of int | Node of (locationLookup<'a>*'a*locationLookup<'a>)

let createLookup someArray = 
    let rec core (a: _ []) lower upper = 
        if upper=lower then Leaf(upper) else
            let div = (upper-lower)/2 + lower
            let value = a.[div]
            let left = core a lower div
            let right = core a (div+1) upper
            Node(left,value,right)
    core someArray 0 (Array.length someArray)
    
let rec lookup lookupTree value =
    match lookupTree with 
    | Leaf(i) -> i
    | Node(l,v,r) -> if value >= v then lookup r value else lookup l value

let findPosition (someArray: 'a []) value =
    let tree = createLookup someArray
    lookup tree value
    
let cdfDifference a b = 
    let aLen = Array.length a |> float
    let bLen = Array.length b |> float
    Array.mapi (fun i e ->  findPosition a e 
                            |> fun s -> abs ( float(s)/aLen - float(i+1)/bLen) 
                            )  b

let simplifiedKS s o =
    let data1 = Array.sort s
    let data2 = Array.sort o
    let max1 = cdfDifference data1 data2 |> Array.max
    let max2 = cdfDifference data2 data1 |> Array.max
    if max1 > max2 then max1 else max2

In [23]:
printf "Accord KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance ksSummStat measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance ksSummStat fakeData1.[i] fakeData2.[i]

printf "Scipy KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance scipyKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance scipyKS fakeData1.[i] fakeData2.[i]

printf "Local KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance simplifiedKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance simplifiedKS fakeData1.[i] fakeData2.[i]

Accord KS calculation
Wall: 15605.594600ms CPU: 15468.750000ms
Wall: 2766.512800ms CPU: 2765.625000ms
Scipy KS calculation
Wall: 705.291000ms CPU: 781.250000ms
Wall: 451.462900ms CPU: 500.000000ms
Local KS calculation
Wall: 97879.194200ms CPU: 98640.625000ms
Wall: 12467.006700ms CPU: 12468.750000ms


In [24]:
printf "Accord: %f\n" <| ksSummStat measurements.[0] measurements.[1]
printf "scipy:  %f\n" <| scipyKS measurements.[0] measurements.[1]
printf "Local:  %f\n" <| simplifiedKS measurements.[0] measurements.[1]

Accord: 0.960469
scipy:  0.960469
Local:  0.960469


## Using inline (successfully)

Simplifying the findPosition function gave us a relatively small speed up. The next thing we try is to use the inline keyword; this instructs the compiler/interpreter to spend extra time on compile to speed up the functions on running. The documentation suggests that it should be used as a last resort, but by using it librally makes the functions comparably fast to scipy.

In [25]:
let inline findPosition (someArray: 'a []) value =
    let rec find lower upper  =
        if lower=upper then upper else
            let div = (upper-lower)/2 + lower    
            if someArray.[div] > value then find lower div 
            else find (div+1) upper 
    find 0 (Array.length someArray) 
    
let inline cdfDifference a b = 
    let aLen = Array.length a |> float
    let bLen = Array.length b |> float
    Array.mapi (fun i e ->  findPosition a e 
                            |> fun s -> abs ( float(s)/aLen - float(i+1)/bLen) 
                            )  b

let inline simplifiedKS s o =
    let data1 = Array.sort s
    let data2 = Array.sort o
    let max1 = cdfDifference data1 data2 |> Array.max
    let max2 = cdfDifference data2 data1 |> Array.max
    if max1 > max2 then max1 else max2

In [26]:
printf "Accord KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance ksSummStat measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance ksSummStat fakeData1.[i] fakeData2.[i]

printf "Scipy KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance scipyKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance scipyKS fakeData1.[i] fakeData2.[i]

printf "Local KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance simplifiedKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance simplifiedKS fakeData1.[i] fakeData2.[i]

Accord KS calculation
Wall: 15529.717700ms CPU: 15546.875000ms
Wall: 2762.428300ms CPU: 2734.375000ms
Scipy KS calculation
Wall: 802.641600ms CPU: 890.625000ms
Wall: 497.312400ms CPU: 578.125000ms
Local KS calculation
Wall: 798.257600ms CPU: 984.375000ms
Wall: 144.340900ms CPU: 203.125000ms


# Final modifications
## Have the benchmarks stopped being useful?

For one of our benchmarks the results are faster, one is slower but the function now is definitely comparable. The final modification we make is to replace the mapi/max operations with a single for loop, using a mutable. 

In other versions of the functions we passed an explict *compare* function to inlined functions, which has been known to improve performance specifically when using inline. We've left it out here as it slowed the functions down, but it may be worth testing in other code.

We finally test outcomes with a slightly larger test set, confirming that all functions give the same outcome.

In [27]:
let inline findPosition (someArray: _ []) value =
    let rec find lower upper  =
        if lower=upper then upper else
            let div = (upper-lower)/2 + lower    
            if someArray.[div] > value then find lower div 
            else find (div+1) upper 
    find 0 (Array.length someArray) 
    
let inline cdfDifference a b = 
    let aLen = Array.length a |> float
    let bLen = Array.length b |> float
    let mutable max = 0.
    for i = 0 to ((Array.length b)-1) do
        let diff = findPosition a b.[i] |> fun s -> abs ( float(s)/aLen - float(i+1)/bLen) 
        if diff > max then max <- diff
    max

let inline simplifiedKS s o =
    let data1 = Array.sort s
    let data2 = Array.sort o
    let max1 = cdfDifference data1 data2 
    let max2 = cdfDifference data2 data1
    if max1 > max2 then max1 else max2

In [28]:
printf "Accord KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance ksSummStat measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance ksSummStat fakeData1.[i] fakeData2.[i]

printf "Scipy KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance scipyKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance scipyKS fakeData1.[i] fakeData2.[i]

printf "Local KS calculation\n"
timeIt <| fun _ -> ignore <| Array.init 100 (fun _ ->  distance simplifiedKS measurements measurements)

timeIt <| fun _ -> for i = 0 to 99 do
                        ignore <| distance simplifiedKS fakeData1.[i] fakeData2.[i]

Accord KS calculation
Wall: 15574.136200ms CPU: 16109.375000ms
Wall: 2753.079300ms CPU: 2750.000000ms
Scipy KS calculation
Wall: 697.065800ms CPU: 796.875000ms
Wall: 581.966700ms CPU: 609.375000ms
Local KS calculation
Wall: 713.561400ms CPU: 875.000000ms
Wall: 101.505200ms CPU: 156.250000ms


In [29]:
printf "Accord: %f\n" <| ksSummStat measurements.[0] measurements.[1]
printf "scipy:  %f\n" <| scipyKS measurements.[0] measurements.[1]
printf "Local:  %f\n" <| simplifiedKS measurements.[0] measurements.[1]
printf "New test\n"
printf "Accord: %f\n" <| ksSummStat measurements.[2] measurements.[1]
printf "scipy:  %f\n" <| scipyKS measurements.[2] measurements.[1]
printf "Local:  %f\n" <| simplifiedKS measurements.[2] measurements.[1]
printf "New test\n"
printf "Accord: %f\n" <| ksSummStat measurements.[0] measurements.[2]
printf "scipy:  %f\n" <| scipyKS measurements.[0] measurements.[2]
printf "Local:  %f\n" <| simplifiedKS measurements.[0] measurements.[2]
printf "New test\n"
printf "Accord: %f\n" <| ksSummStat measurements.[1] measurements.[0]
printf "scipy:  %f\n" <| scipyKS measurements.[1] measurements.[0]
printf "Local:  %f\n" <| simplifiedKS measurements.[1] measurements.[0]
printf "New test\n"
printf "Accord: %f\n" <| ksSummStat measurements.[2] measurements.[0]
printf "scipy:  %f\n" <| scipyKS measurements.[2] measurements.[0]
printf "Local:  %f\n" <| simplifiedKS measurements.[2] measurements.[0]

Accord: 0.960469
scipy:  0.960469
Local:  0.960469
New test
Accord: 0.762915
scipy:  0.762915
Local:  0.762915
New test
Accord: 0.997738
scipy:  0.997738
Local:  0.997738
New test
Accord: 0.960469
scipy:  0.960469
Local:  0.960469
New test
Accord: 0.997738
scipy:  0.997738
Local:  0.997738


# More realistic benchmarks

The results are positive, but given the small change difference between our benchmark sets the fear might be that they are unnatural and further optimisation might mislead us. To get more realistic results below we have taken a minimal set of functions from the [main notebook](FSharpSimulator.ipynb) and timed both a set of particle searches, and the relative time taken on calculating distance.

In [30]:
let gamma mean shape rng =
    let invscale = shape/mean
    fun _ -> MathNet.Numerics.Distributions.Gamma.Sample(rng, shape, invscale)
    
let gammaDelay delay mean shape rng = 
    let gFunc = gamma (mean-delay) shape rng
    fun _ -> delay + (gFunc ())
    
let gammaFracDelay fracDelay mean shape rng = 
    //Expressing the delay as a fraction of the mean might be a better way..
    let delay = mean*fracDelay
    gammaDelay delay mean shape rng
    
let noNoise =
    fun _ -> 0.
    
let piedrafitaNoise (rng: System.Random) =
    let sqrt12 = 12.**0.5
    fun _ -> (rng.NextDouble()-0.5)*0.06*sqrt12
    //noiseTerms = np.random.uniform(-0.5,0.5,n)*math.sqrt(12)*0.06

let randomSample (rng: System.Random) (a: 'a []) =
    //Randomly select and return an element of the input array
    a.[rng.Next((Array.length a))]

let invertedWeightRandomSample (rng: System.Random) weights (a: 'a []) =
    //Input will be distances-> big distances are worse
    //The weight/probability relationship is therefore assumed to be smaller -> more probable
    //Invert the weights, normalise, and convert to a cdf
    let invertedW = Array.map (fun i -> 1./i) weights
    let total = Array.sum invertedW
    //let prob = Array.map (fun i -> i/total) invertedW
    //printf "Probability\n%A\n" prob
    let cdf = Array.scan (fun totalP i -> totalP+(i/total) ) 0. invertedW
    //printf "CDF\n%A\n" cdf
    fun _ ->
        let num = rng.NextDouble()
        Array.findIndex (fun p -> p>num) cdf
        |> fun i -> a.[i-1]

let weightedRandomSample (rng: System.Random) weights (a: 'a []) =
    //Input will be likelihoods-> big likelihoods are better
    //Take the weights, normalise, and convert to a cdf
    let total = Array.sum weights
    let cdf = Array.scan (fun totalP i -> totalP+(i/total) ) 0. weights
    fun _ ->
        let num = rng.NextDouble()
        Array.findIndex (fun p -> p>num) cdf
        |> fun i -> a.[i-1]

let rec diluteEngine maxtime div noise times dilutions lasttime label =
    if lasttime > maxtime then 
        List.map2 (fun t d -> (t,d)) (lasttime::times) (label::dilutions) |> List.rev
    else
        let dt = div ()
        let lasttime' = lasttime + dt
        let dLabel = 0.5 + (noise ())
        let label' = dLabel*label
        diluteEngine maxtime div noise (lasttime::times) (label::dilutions) lasttime' label'

let rec findObservation timePoints simulation acc prevDilution =
    match timePoints, simulation with
    | [],_ -> List.rev acc //all done
    | _, [] -> failwith "Run out of simulations"
    | t'::otherTimes, (simTime,d)::otherMeasurements -> 
        if simTime > t' then
            //The next timepoint has past- return the *previous* dilution
            let pDil = match prevDilution with
                        | Some(oldD) -> oldD
                        | None -> failwith "Requested timepoint occurs before t=0"
            //How many timepoints have passed? Need to count the timepoints and add the previous label that number of times
            //let acc', remainingTime = peek otherTimes t' (pDil::acc) pDil
            findObservation otherTimes simulation (pDil::acc) prevDilution
        else 
            //timepoint has not arrived, move on
            findObservation timePoints otherMeasurements acc (Some(d))
            
let dilution noise div (rng: System.Random) timePoints initialLabel randomiseTime1 = 
    let lastTime = List.max timePoints
    let experiment = if randomiseTime1 then
                        //randomise the initial timepoint
                        let t1 = ( div () ) * rng.NextDouble()
                        let dLabel = (+) 0.5 <| noise ()
                        let label = randomSample rng initialLabel
                        let label' = dLabel* label
                        diluteEngine lastTime div noise [0.] [label] t1 label'
                     else
                         //don't bother
                         diluteEngine lastTime div noise [] [] 0. <| randomSample rng initialLabel
    //for e in experiment do
        //let t,d = e
        //printf "%f\t%f\n" t (Math.Log(d,2.))
    try
        findObservation timePoints experiment [] None |> Array.ofList
    with
        | Failure msg -> failwithf "%s\n%A\n" msg experiment

let dilutionExperiment noise div rng timePoints initialLabel sampleSize randomInit =
        Array.init sampleSize <| fun _ -> dilution noise div rng timePoints initialLabel randomInit

let transpose (a: 'a [] []) = 
    let x,y = ((Array.length a), (Array.length a.[0]))
    Array.init y (fun yi -> Array.init x (fun xi -> a.[xi].[yi]))

let uniformPrior min max (rng: System.Random) =
    fun _ -> min + (max-min)*rng.NextDouble()
    
type modelRange =
    {
    fracMax : float
    fracMin : float
    shapeMax : float
    shapeMin : float
    }

type modelPriors = 
    {
    fracDelay : unit->float
    log2Shape : unit->float
    }

let rangeToPrior r rng =
    {
    fracDelay = uniformPrior r.fracMin r.fracMax rng
    log2Shape = uniformPrior r.shapeMin r.shapeMax rng
    }

type modelSelection =
    {
    fracDelaySel : float
    log2ShapeSel : float
    }

type modelFixed = 
    {
    mean : float
    initialLabel : float []
    noise : System.Random -> unit -> float
    sampleSize : int
    timePoints : float list
    rng : System.Random
    unknownRanges : modelRange
    randomiseInit : bool
    }

let modelInterface fixPa priors spareRng = 
    let rng = match spareRng with 
              | Some(r) ->r
              | None -> fixPa.rng
    let delay = priors.fracDelay ()
    let log2Shape = priors.log2Shape ()
    let selection = {fracDelaySel=delay;log2ShapeSel=log2Shape}
    let div = gammaFracDelay delay fixPa.mean (2.**log2Shape) rng
    let noise = fixPa.noise rng
    let result = dilutionExperiment noise div rng fixPa.timePoints fixPa.initialLabel fixPa.sampleSize fixPa.randomiseInit
                    |> Array.map (fun m -> Array.map (fun i -> Math.Log(i,2.) ) m)
    (selection,(transpose result))
    
let modelInterfaceParticle fixPa rng p  = 
    let noise = fixPa.noise rng
    let div = gammaFracDelay p.fracDelaySel fixPa.mean (2.**p.log2ShapeSel) rng
    let result = dilutionExperiment noise div rng fixPa.timePoints fixPa.initialLabel fixPa.sampleSize fixPa.randomiseInit
                    |> Array.map (fun m -> Array.map (fun i -> Math.Log(i,2.) ) m)
    (p,(transpose result)) 

let ksSummStat s o =
    Accord.Statistics.Testing.TwoSampleKolmogorovSmirnovTest(s,o).Statistic
    
let quantileSummStat q s o =
    let sq = Accord.Statistics.Measures.Quantiles(s,q)
    let oq = Accord.Statistics.Measures.Quantiles(o,q)
    Array.map2 (fun a b -> abs(a-b)) sq oq
    |> Array.sum

let distance (summStat: 'a [] -> 'a [] -> float) sim measurements = 
    Array.map2 (fun s o -> summStat s o )  sim measurements
    |> Array.sum

type Space = Global | Local of float

type inference = {  populations : int
                    popSize : int
                    maxAttempts : int
                    epsilonMultiplier : float
                    summaryStatistic : float [] -> float [] -> float
                    transition : Space
                    weightPreviousPopulation : bool //Weight previous population by distance to speed up search
                    initialEpsilon : float option
                    }

//Threadsafe bool object for checking other results
type tsBool() =
    let v = ref 0
    member this.contents with 
                                get() = (System.Threading.Interlocked.CompareExchange(v, 1, 1) = 1) 
                                and set(value:bool) = if value then
                                                            ignore( System.Threading.Interlocked.CompareExchange(v, 1, 0) )
                                                        else 
                                                            ignore( System.Threading.Interlocked.CompareExchange(v, 0, 1) )
                                                            
let randomWalk particles rng =
    let samples = Array.map (fun c -> [|c.fracDelaySel;c.log2ShapeSel|]) particles
    let kernel = new Accord.Statistics.Distributions.DensityKernels.GaussianKernel(2)
    let dist = new Accord.Statistics.Distributions.Multivariate.MultivariateEmpiricalDistribution(kernel, samples)
    let cov = dist.Covariance
    let cho = new Accord.Math.Decompositions.CholeskyDecomposition(cov)
    let L =     cho.LeftTriangularFactor
    //cho.LeftTriangularFactor
    fun _ ->
        let rand = Array.init 2 (fun _ -> MathNet.Numerics.Distributions.Normal.Sample(rng,0.,1.) )
        [| 
            L.[0,0] * rand.[0] + L.[0,1] * rand.[1] ;
            L.[1,0] * rand.[0] + L.[1,1] * rand.[1]
            |]

let getLocalCov number all particle =
    let neighbours = Array.sortBy (fun p -> (pown (p.fracDelaySel-particle.fracDelaySel) 2) + (pown (p.log2ShapeSel-particle.log2ShapeSel) 2) ) all
                     |> fun sortedNeighbours -> Array.init number (fun i -> sortedNeighbours.[i])
    let samples = Array.map (fun c -> [|c.fracDelaySel;c.log2ShapeSel|]) neighbours
    let kernel = new Accord.Statistics.Distributions.DensityKernels.GaussianKernel(2)
    let dist = new Accord.Statistics.Distributions.Multivariate.MultivariateEmpiricalDistribution(kernel, samples)
    let cov = dist.Covariance
    let cho = new Accord.Math.Decompositions.CholeskyDecomposition(cov)
    particle.ToString(), cho.LeftTriangularFactor

let localRandomWalk populationFraction particles rng =
    //Find the nearest neighbours for each particle
    let numberNeighbours = float(Array.length particles) * populationFraction |> int
    let local = getLocalCov numberNeighbours particles
    let environments = Array.map local particles
    let dict = new System.Collections.Generic.Dictionary<string,float[,]>()
    Array.iter (fun i -> dict.Add(i) ) environments
    fun p ->
        match dict.TryGetValue p with
        | false, _ -> failwithf "Particle %s not in original population" p
        | true, L ->    let rand = Array.init 2 (fun _ -> MathNet.Numerics.Distributions.Normal.Sample(rng,0.,1.) )
                        [| 
                        L.[0,0] * rand.[0] + L.[0,1] * rand.[1] ;
                        L.[1,0] * rand.[0] + L.[1,1] * rand.[1]
                        |]

let kdeWeights particles =
    let samples = Array.map (fun c -> [|c.fracDelaySel;c.log2ShapeSel|]) particles
    let kernel = new Accord.Statistics.Distributions.DensityKernels.GaussianKernel(2)
    let dist = new Accord.Statistics.Distributions.Multivariate.MultivariateEmpiricalDistribution(kernel, samples)
    Array.map (fun value -> dist.ProbabilityDensityFunction(value)) samples

let perturb inputRanges (walk: string->float[]) particle =
    let randWalk = walk (particle.ToString())
    let dDelay = 
        let r = randWalk.[0] + particle.fracDelaySel
        //MathNet.Numerics.Distributions.Normal.Sample(particle.fracDelaySel,)
        if r > inputRanges.fracMax then 2.*inputRanges.fracMax-r
        elif r < inputRanges.fracMin then 2.*inputRanges.fracMin-r
        else r
    let dShape = 
        let r = randWalk.[1] + particle.log2ShapeSel
        if r > inputRanges.shapeMax then 2.*inputRanges.shapeMax-r
        elif r < inputRanges.shapeMin then 2.*inputRanges.shapeMin-r
        else r
    {particle with 
        fracDelaySel=dDelay; 
        log2ShapeSel=dShape;
        }

let rec findParticleTime lineTime (failedPrevious: tsBool) previousPopulation walk selectFunc eps fixPa infP attempts spareRng=
   if failedPrevious.contents then None else
       let rng = match spareRng with
                   | Some(r) -> r
                   | None -> fixPa.rng
       if attempts = infP.maxAttempts then
           sprintf "findParticle had too many attempts to finish\n" |> Display
           failedPrevious.contents <- true
           None
       else
           //Picking
           let pi = selectFunc previousPopulation |> perturb fixPa.unknownRanges walk
           //let p,d = selectFunc previousPopulation
                       //|> perturb fixPa.unknownRanges 0.5 0.5 rng
           //Simulation
           let p,sim = modelInterfaceParticle fixPa rng pi
           let d = timeStore lineTime (fun _ -> distance infP.summaryStatistic sim measurements)

           if d < eps then Some(p,d) else findParticleTime lineTime failedPrevious previousPopulation walk selectFunc eps fixPa infP (attempts+1) spareRng


In [31]:
let rng = System.Random(1982)
let searchRange = 
    {
        fracMin = 0.
        fracMax = 1.
        shapeMin = 0.
        shapeMax = 6.
    }
 
let fixedParameters =
    {
        mean = (7./3.);
        initialLabel = i0;
        noise = piedrafitaNoise 
        sampleSize = 1000
        timePoints = [7.;12.;18.]
        rng = rng
        unknownRanges=searchRange
        randomiseInit = true
    }

let fail = tsBool()
let lineTime = ref 0.
let inferenceP =     { 
        populations = 4; //4 works
        popSize = 200; //50 works
        maxAttempts = 1000; //1000 for benchmakrs
        epsilonMultiplier = 0.98 //0.99 works
        summaryStatistic = ksSummStat//quantileSummStat [|0.025;0.25;0.5;0.75;0.975|]
        transition = Global 
        weightPreviousPopulation = false
        initialEpsilon = Some(0.5)
    }
let selectFunc r =    let priors = rangeToPrior fixedParameters.unknownRanges r
                      let delay = priors.fracDelay ()
                      let log2Shape = priors.log2Shape ()
                      fun _  -> {fracDelaySel=delay;log2ShapeSel=log2Shape}
let walk = fun _ -> [|0.;0.|]
let makePop = fun r -> findParticleTime lineTime fail () walk (selectFunc r) 0.5 fixedParameters inferenceP 0 (Some(r))
let ip = timeIt (fun _ -> makePop (System.Random(1887)) )
printf "Time spent measuring distances = %fms\n" !lineTime

"findParticle had too many attempts to finish
"

Wall: 102041.012900ms CPU: 102125.000000ms
Time spent measuring distances = 99553.950000ms


In [32]:
let rng = System.Random(1982)
let searchRange = 
    {
        fracMin = 0.
        fracMax = 1.
        shapeMin = 0.
        shapeMax = 6.
    }
 
let fixedParameters =
    {
        mean = (7./3.);
        initialLabel = i0;
        noise = piedrafitaNoise 
        sampleSize = 1000
        timePoints = [7.;12.;18.]
        rng = rng
        unknownRanges=searchRange
        randomiseInit = true
    }

let fail = tsBool()
let lineTime = ref 0.
let inferenceP =     { 
        populations = 4; //4 works
        popSize = 200; //50 works
        maxAttempts = 1000; //1000 for benchmakrs
        epsilonMultiplier = 0.98 //0.99 works
        summaryStatistic = scipyKS//quantileSummStat [|0.025;0.25;0.5;0.75;0.975|]
        transition = Global 
        weightPreviousPopulation = false
        initialEpsilon = Some(0.5)
    }
let selectFunc r =    let priors = rangeToPrior fixedParameters.unknownRanges r
                      let delay = priors.fracDelay ()
                      let log2Shape = priors.log2Shape ()
                      fun _  -> {fracDelaySel=delay;log2ShapeSel=log2Shape}
let walk = fun _ -> [|0.;0.|]
let makePop = fun r -> findParticleTime lineTime fail () walk (selectFunc r) 0.5 fixedParameters inferenceP 0 (Some(r))
let ip = timeIt (fun _ -> makePop (System.Random(1887)) )
printf "Time spent measuring distances = %fms\n" !lineTime

"findParticle had too many attempts to finish
"

Wall: 41478.051200ms CPU: 44484.375000ms
Time spent measuring distances = 8515.797600ms


In [33]:
let rng = System.Random(1982)
let searchRange = 
    {
        fracMin = 0.
        fracMax = 1.
        shapeMin = 0.
        shapeMax = 6.
    }
 
let fixedParameters =
    {
        mean = (7./3.);
        initialLabel = i0;
        noise = piedrafitaNoise 
        sampleSize = 1000
        timePoints = [7.;12.;18.]
        rng = rng
        unknownRanges=searchRange
        randomiseInit = true
    }

let fail = tsBool()
let lineTime = ref 0.
let inferenceP =     { 
        populations = 4; //4 works
        popSize = 200; //50 works
        maxAttempts = 1000; //1000 for benchmakrs
        epsilonMultiplier = 0.98 //0.99 works
        summaryStatistic = simplifiedKS//quantileSummStat [|0.025;0.25;0.5;0.75;0.975|]
        transition = Global 
        weightPreviousPopulation = false
        initialEpsilon = Some(0.5)
    }
let selectFunc r =    let priors = rangeToPrior fixedParameters.unknownRanges r
                      let delay = priors.fracDelay ()
                      let log2Shape = priors.log2Shape ()
                      fun _  -> {fracDelaySel=delay;log2ShapeSel=log2Shape}
let walk = fun _ -> [|0.;0.|]
let makePop = fun r -> findParticleTime lineTime fail () walk (selectFunc r) 0.5 fixedParameters inferenceP 0 (Some(r))
let ip = timeIt (fun _ -> makePop (System.Random(1887)) )
printf "Time spent measuring distances = %fms\n" !lineTime

"findParticle had too many attempts to finish
"

Wall: 18675.888400ms CPU: 20265.625000ms
Time spent measuring distances = 1093.701300ms


# Conclusions

Identifying one specific function in our workflow and optimising made a massive difference to performance. Simple modifications to the code, tested at each point allowed us to write substantially faster code eventually. This was further improved by using the *inline* keyword to move work to the compiler/interpreter and away from individual runs.